# Patent Analysis

In [17]:
%%capture
# !pip install -r requirements.txt

## Importing Libraries

In [54]:
%%capture
import sys
import importlib
sys.path.append('src')


import preprocess_data
import qaagent

importlib.reload(preprocess_data)
importlib.reload(qaagent)


In [19]:

import nltk
nltk.download('all', quiet=True)

True

## Downloading patents and preprocessing

In [20]:
year = 2023
month = 1
day = 5

In [21]:
saved_patent_names = preprocess_data.parse_and_save_patents(year, month, day, True)

### Downloading weekly patent files...
Building the URL...
URL constructed: https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2023/ipa230105.zip


Requesting the file...
File retrieved successfully. Starting download...
File downloaded successfully. Starting extraction...
File extracted successfully.
ZIP file c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\patents.zip deleted after extraction.
### Extracting individual patents...
Locating the patent file...
Reading the patent file...
Splitting the XMl file into individual XMLs...
Total patents found: 7823
Creating directory to store individual patents...
Writing individual patents to separate txt files...
Patent 20230000001 does not belong to section 'C'. Skipping this patent.
Patent 20230000002 does not belong to section 'C'. Skipping this patent.
Patent 20230000003 does not belong to section 'C'. Skipping this patent.
Patent 20230000004 does not belong to section 'C'. Skipping this patent.
Patent 20230000005 does not belong to section 'C'. Skipping this patent.
Patent 20230000006 does not belong to section 'C'. Skipping this patent.
Patent 20230000007 does not belon

## Loading data and querying sample file


In [50]:
Prompt = """
Carefully review the given context and extract as much information as possible about measurements, their values, and units mentioned in the context. 

For each measurement, please provide the following details:
- The context in which the measurement is mentioned. (Measurement Context)
- The specific value or range given for the measurement. (Measured Value)
- The unit of the measurement, if provided. (Unit)
- What the measurement is referring to, if it can be determined from the context (e.g., a physical property, a characteristic, a specific part of a device, etc.). (Refers To)


Format your response in a structured JSON-like format, as follows:

{
  "Content": [
    {
      "Measurement Context": "context",
      "Measured Value": "value",
      "Unit": "unit",
      "Refers To": "reference"
    },
    // ... additional measurements, if present
  ]
}

If multiple measurements are present in the text, each should be listed as a separate object within the "Content" array.

Try to provide as complete and accurate information as possible. If certain details cannot be determined, indicate this by writing "Unknown".

Do not print any response other than the json.
"""



In [42]:
import json
import os

# Creating an empty dictionary to store the JSON data
json_files = {}

# Specifying the path to the JSON files
folder_path = os.path.join(
        os.getcwd(),
        "output"
    )


# Iterating through every file in the directory
for file_name in os.listdir(folder_path):
    # Checking if the current file is a JSON file
    if file_name.endswith('.json'):
        # Defining the path to the file
        file_path = os.path.join(folder_path, file_name)

        # Opening the JSON file
        with open(file_path, 'r') as file:
            # Loading the JSON data from the file
            data = json.load(file)

            # Adding the JSON data to the dictionary
            # The key is the file name split at the first "-" and taking the first part
            key_name = file_name.split('-')[0]
            json_files[key_name] = data


# Printing the dictionary to check if the data was loaded correctly
poc = []
for key in json_files:
    print(f"{key}: {json_files[key]}")
    poc.append(key)



8.json: {'Patent Identifier': 'US 20230001023A1', 'Content': [{'Measurement Context': 'bolus of a renal injury inducer', 'Measured Value': 'dosage sufficiently high', 'Unit': 'Unknown', 'Refers To': 'dosage'}, {'Measurement Context': 'albumin creatinine ratio (ACR)', 'Measured Value': 'Unknown', 'Unit': 'Unknown', 'Refers To': 'measurement of renal injury'}, {'Measurement Context': 'transcutaneous fluorescence', 'Measured Value': 'Unknown', 'Unit': 'Unknown', 'Refers To': 'measurement of renal injury'}, {'Measurement Context': 'degree of renal injury and/or alteration of GFR', 'Measured Value': 'Unknown', 'Unit': 'Unknown', 'Refers To': 'measurement of renal injury'}]}
US20230001042A1: {'Patent Identifier': 'Unknown', 'Content': [{'Measurement Context': 'compound P-6', 'Measured Value': '23,350', 'Unit': 'Unknown', 'Refers To': 'weight average molecular weight (Mw)'}, {'Measurement Context': 'compound P-6', 'Measured Value': '6.5', 'Unit': 'Unknown', 'Refers To': 'polydispersity (Mw/Mn

In [43]:
indices = []
for poc_item in poc:
    for i, patent_path in enumerate(saved_patent_names):
        if poc_item in patent_path:
            indices.append(i)
print(indices)


[96, 130, 227, 497, 502, 503, 507, 602]


In [55]:
raw, output = qaagent.call_QA_to_json(Prompt, year, month, day, saved_patent_names, count=96)

Loading documents from: c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\ipa230105\US20230001042A1-20230105.XML.txt
Generating embeddings and persisting...
Running retrieval chain...
Writing the output to a file...
Call to 'call_QA_to_json' completed.


In [58]:
for idx in indices:
    raw, output = qaagent.call_QA_to_json(Prompt, year, month, day, saved_patent_names, count=idx)
    print(raw)

Loading documents from: c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\ipa230105\US20230001042A1-20230105.XML.txt
Generating embeddings and persisting...
Running retrieval chain...
Writing the output to a file...
Call to 'call_QA_to_json' completed.
[Document(page_content="-patent_id: 20230001042 -file_id: US20230001042A1-20230105.XML -full text: TECHNICAL FIELD The present invention relates to a medical suture thread used for medical action such as a surgery or injury treatment. BACKGROUND ART Typically, a medical suture thread used for medical action such as a surgery or injury treatment has been used. Of this medical suture thread, an outer surface is coated with various resin materials for the purpose of improving operability in suturing and slidability or slide-down performance (the easiness of moving a knot upon thread tying) on a sutured portion. For example, Patent Literature 1 below discloses that a suture thread is coated with 2-methacryloyloxyethyl phosphorylcho

## Updating requirments file

In [ ]:
# !pip freeze > requirements.txt